# Dreambooth

## Build Environment

In [ ]:
%cd /workspace/Person-Plus-Style-Stable-Diffusion/

/workspace/Person-Plus-Style-Stable-Diffusion


In [7]:
%ls

JupyterNotebookHelpers/  dreambooth_runpod_joepenna.ipynb  merge_embeddings.py
LICENSE                  environment.yaml                  models/
README.md                evaluation/                       prune_ckpt.py
assets/                  gdrive                            readme-images/
configs/                 ldm/                              scripts/
data/                    main.py                           setup.py


In [8]:
# BUILD ENV
!pip install omegaconf
!pip install einops
!pip install pytorch-lightning==1.6.5
!pip install test-tube
!pip install transformers
!pip install kornia
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install setuptools
!pip install pillow==9.0.1
!pip install torchmetrics==0.6.0
!pip install -e .
!pip install protobuf
!pip install gdown
!pip install -qq diffusers["training"]==0.3.0 transformers ftfy
!pip install huggingface_hub
!pip install captionizer==1.0.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 48.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=350615c220d525eca3578f47e39cc58a098d511fb7f312dcf64b45152e65660c
  Stored in directory: /root/.cache/pip/wheels/12/93/dd/1f6a127edc45659556564c5730f6d4e300888f4bca2d4c5a88
Successfully built antlr4-python3-runtime

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 225.9 kB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 kB 5.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Download the 1.5 model from hugging face
You can also provide your own v1.* model for training by uploading it and renaming it to "model.ckpt".  It should be in the same directory as dreambooth_runpod_joepenna.ipynb

In [9]:
# Download the 1.5 sd model
from IPython.display import clear_output
from huggingface_hub import hf_hub_download

downloaded_model_path = hf_hub_download(
 repo_id="PiyarSquare/sd_asim_simpsons",
 filename="asim_v1.ckpt"
)

# Move the sd_v1-5_vae.ckpt to the root of this directory as "model.ckpt"
actual_locations_of_model_blob = !readlink -f {downloaded_model_path}
!mv {actual_locations_of_model_blob[-1]} model.ckpt
clear_output()
print("✅ model.ckpt successfully downloaded")

✅ model.ckpt successfully downloaded


# Download pre-generated regularization images
We've created the following image sets

* `man_euler` - provided by Niko Pueringer (Corridor Digital) - euler @ 40 steps, CFG 7.5
* `man_unsplash` - pictures from various photographers
* `person_ddim` - provided by David Bielejeski - ddim @ 50 steps, CFG 10.0
* `woman_ddim` - provided by David Bielejeski - ddim @ 50 steps, CFG 10.0
* `artstyle` - provided by Hackmans - ddim @ 50 steps, CFG 10.0

`person_ddim` is recommended for training people


In [ ]:
#Download Regularization Images

dataset="person_ddim" #@param ["man_euler", "man_unsplash", "person_ddim", "woman_ddim", "artstyle"]
!git clone https://github.com/djbielejeski/Stable-Diffusion-Regularization-Images-{dataset}.git

!mkdir -p regularization_images/{dataset}

# rename images & hide output
!mv -v Stable-Diffusion-Regularization-Images-{dataset}/{dataset}/*.* regularization_images/{dataset} > /dev/null

# Upload your training images
Upload 10-20 images of someone to

```
/workspace/Person-Plus-Style-Stable-Diffusion/training_images
```

WARNING: Be sure to upload an *even* amount of images, otherwise the training inexplicably stops at 1500 steps.

*   2-3 full body
*   3-5 upper body
*   5-12 close-up on face

The images should be:

- as close as possible to the kind of images you're trying to make

In [1]:
!mkdir training_images

In [1]:
#@title Download and check the images you have just added
import os
import requests
from io import BytesIO
from PIL import Image
from smart_crop import *
import tqdm
# crop images

for filename in tqdm(os.listdir('training_images'), bar_format='  |{bar:15}| {n_fmt}/{total_fmt} Uploaded'):
    extension = filename.split(".")[-1]
    identifier=filename.split(".")[0]
    new_path_with_file = os.path.join('training_images', filename)
    file = Image.open('training_images'+"/"+filename)
    width, height = file.size
    if file.size !=(512, 512):
        image=crop_image(file, 512)
        if extension.upper()=="JPG" or extension.upper()=="jpg":
            image[0] = image[0].convert("RGB")
            image[0].save(new_path_with_file, format="JPEG", quality = 100)
        else:
            image[0].save(new_path_with_file, format=extension.upper())

ModuleNotFoundError: No module named 'cv2'

## Training

If training a person or subject, keep an eye on your project's `logs/{folder}/images/train/samples_scaled_gs-00xxxx` generations.

If training a style, keep an eye on your project's `logs/{folder}/images/train/samples_gs-00xxxx` generations.

In [ ]:
# Training

# This isn't used for training, just to help you remember what your trained into the model.
project_name = "asimrhaenyra"

# MAX STEPS
# How many steps do you want to train for?
max_training_steps = 1010

# Match class_word to the category of the regularization images you chose above.
class_word = "person" # typical uses are "man", "person", "woman"

# If you are training a person's face, set this to True
i_am_training_a_persons_face = True

flip_p_arg = 0.0 if i_am_training_a_persons_face else 0.5

# This is the unique token you are incorporating into the stable diffusion model.
token = "asimrhaenyra"

# 0 Saves the checkpoint when max_training_steps is reached.
# 250 saves the checkpoint every 250 steps as well as when max_training_steps is reached.
save_every_x_steps = 0

reg_data_root = "/workspace/Person-Plus-Style-Stable-Diffusion/regularization_images/" + dataset

!rm -rf training_images/.ipynb_checkpoints
!python "main.py" \
 --base "configs/stable-diffusion/v1-finetune_unfrozen.yaml" \
 -t \
 --actual_resume "model.ckpt" \
 --reg_data_root "{reg_data_root}" \
 -n "{project_name}" \
 --gpus 0, \
 --data_root "/workspace/Person-Plus-Style-Stable-Diffusion/training_images" \
 --max_training_steps {max_training_steps} \
 --class_word "{class_word}" \
 --token "{token}" \
 --no-test \
 --flip_p {flip_p_arg} \
 --save_every_x_steps {save_every_x_steps}


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
Global seed set to 23
Running on GPUs 0,
Loading model from model.ckpt
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 64, 64) = 16384 dimensions.
making attention of type 'vanilla' with 512 in_channels
Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.5.mlp.fc2.bias', 'vision_model.encoder.layers.4.mlp.fc2.bias', 'vision_model.encoder.layers.15.mlp.fc2.bias', 'vision_model.encoder.layers.19.self_attn.v_proj.bias', 'vision_model.encoder.layers.16.mlp.fc2.bias', 'vision_model.encoder.layers.13.mlp.fc2.weight', 'vision_mode

## Copy and name the checkpoint file(s)

In [ ]:
import re

training_images = !find training_images/*
date_string = !date +"%Y-%m-%dT%H-%M-%S"

if save_every_x_steps == 0:
 # Copy the checkpoint into our `trained_models` folder
 directory_paths = !ls -d logs/*
 last_checkpoint_file = directory_paths[-1] + "/checkpoints/last.ckpt"
 file_name = date_string[-1] + "_" + \
             project_name + "_" + \
             str(len(training_images)) + "_training_images_" + \
             str(max_training_steps) + "_max_training_steps_" + \
             token + "_token_" + \
             class_word + "_class_word.ckpt"

 file_name = file_name.replace(" ", "_")

 !mkdir -p trained_models
 !mv {last_checkpoint_file} trained_models/{file_name}

 print("Download your trained model from trained_models/" + file_name + " and use in your favorite Stable Diffusion repo!")
else:
 directory_paths = !ls -d logs/*
 checkpoints_directory = directory_paths[-1] + "/checkpoints/trainstep_checkpoints"
 file_paths = !ls -d "{checkpoints_directory}"/*

 for i, original_file_name in enumerate(file_paths):
  # Remove the "epoch=000000-step=0000" text
  steps = re.sub(checkpoints_directory + "/epoch=\d{6}-step=0*", "", original_file_name)

  # Remove the .ckpt
  steps = steps.replace(".ckpt", "")

  # Setup the filename
  file_name = date_string[-1] + "_" + \
                  project_name + "_" + \
                  str(len(training_images)) + "_training_images_" + \
                  str(steps) + "_training_steps_" + \
                  token + "_token_" + \
                  class_word + "_class_word.ckpt"

  file_name = file_name.replace(" ", "_")

  # Make the directory and move the files into it.
  !mkdir -p trained_models
  !mv {original_file_name} trained_models/{file_name}

 print("Download your trained models from the 'trained_models' folder and use in your favorite Stable Diffusion repo!")

# Optional - Upload to google drive
* run the following commands in a new `terminal` in the `Dreambooth-Stable-Diffusion` directory
* `chmod +x ./gdrive`
* `./gdrive about`
* `paste your token here after navigating to the link`
* `./gdrive upload trained_models/{file_name.ckpt}`

# Big Important Note!

The way to use your token is `<token> <class>` ie `joepenna person` and not just `joepenna`

## Generate Images With Your Trained Model!

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples 1 \
 --n_iter 4 \
 --scale 7.0 \
 --ddim_steps 50 \
 --ckpt "/workspace/Person-Plus-Style-Stable-Diffusion/trained_models/{file_name}" \
 --prompt "{token} {class_word} as a masterpiece portrait painting by John Singer Sargent in the style of Rembrandt"